In [1]:
import pickle
import traceback

In [2]:
import pandas as pd
import time
import re
from IPython.display import clear_output

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from agenda_tools.get_schedule import remove_blank_spaces
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities


from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementNotInteractableException
from concurrent.futures import ThreadPoolExecutor

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
# options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:107.0) Gecko/20100101 Firefox/107.0")
# options.add_argument('--headless')
options.add_argument("--incognito")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.google.com")

[WDM] - Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 6.98M/6.98M [00:12<00:00, 575kB/s]


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
# options.add_argument('--headless')
options.add_argument("--incognito")

prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:107.0) Gecko/20100101 Firefox/107.0")


capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options, desired_capabilities=capa)
driver.get("https://www.google.com")
wait = WebDriverWait(driver, 20)

SessionNotCreatedException: Message: session not created
from disconnected: received Inspector.detached event
  (Session info: chrome=105.0.5195.102)
Stacktrace:
#0 0x559be563e693 <unknown>
#1 0x559be5437b0a <unknown>
#2 0x559be54253b8 <unknown>
#3 0x559be5424f8f <unknown>
#4 0x559be5424389 <unknown>
#5 0x559be5422fd0 <unknown>
#6 0x559be54233b2 <unknown>
#7 0x559be54228b1 <unknown>
#8 0x559be542df37 <unknown>
#9 0x559be5422853 <unknown>
#10 0x559be5423d70 <unknown>
#11 0x559be5422fd0 <unknown>
#12 0x559be54233b2 <unknown>
#13 0x559be54228b1 <unknown>
#14 0x559be5429764 <unknown>
#15 0x559be5422853 <unknown>
#16 0x559be5423d70 <unknown>
#17 0x559be5422fd0 <unknown>
#18 0x559be54233b2 <unknown>
#19 0x559be54228b1 <unknown>
#20 0x559be541d40a <unknown>
#21 0x559be5422853 <unknown>
#22 0x559be5422764 <unknown>
#23 0x559be54221e0 <unknown>
#24 0x559be5439072 <unknown>
#25 0x559be5494ee7 <unknown>
#26 0x559be548d743 <unknown>
#27 0x559be5463533 <unknown>
#28 0x559be5464715 <unknown>
#29 0x5

In [5]:
def scroll():
    previous_heigth = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(5)
        new_heigth = driver.execute_script('return document.body.scrollHeight')
        if new_heigth == previous_heigth:
            break
        previous_heigth = new_heigth

In [6]:
def search_google(driver,cif):
    # BORRAR LA ANTERIOR BUSQUEDA
    try:
        driver.find_element('xpath', '//span[@class="ExCKkf z1asCe rzyADb"]').click()
    except ElementNotInteractableException:
        pass
    
    search = f'einforma {cif}'
    try:
        driver.find_element('xpath', '//input[@class="gLFyf"]').send_keys(search)

        driver.find_element('xpath', '//input[@class="gLFyf"]').send_keys(Keys.ENTER)
        return True
    except:
        print('ERROR')
        return False

In [7]:
def click_page(driver, wait):
    ispage = False
    n = 1
    while not ispage:
        try:
            wait.until(EC.presence_of_element_located((By.XPATH, f'//div[@class="v7W49e"]/div[position()={n}]//cite')))
            dominio = driver.find_element('xpath', f'//div[@class="v7W49e"]/div[position()={n}]//cite').text
        except:
            dominio = ''
        if 'einforma.com' in dominio:
            ispage = True
            return driver.find_element('xpath', f'//div[@class="v7W49e"]/div[position()={n}]//h3/../../a').get_attribute('href')
#             return True
        if n >= 4:
            return False
        n += 1
        #assert n < 5, 'Bucle infinito'    

# PROCESS

In [30]:
df = pd.read_csv('cif_datoscif_galicia.csv')

In [31]:
list_cif = df.Cif 

In [32]:
list_cif

0       B15494313
1       B70240049
2       B15452048
3       B70297775
4       B70542576
          ...    
4267    B15894280
4268    B70453287
4269    B70478680
4270    B70213863
4271    B81004525
Name: Cif, Length: 4272, dtype: object

In [10]:
driver.get('https://www.google.com/search?q=einforma+B70341466&ei=yGZ6Y7eoM9mYwt0PpJKwYA&ved=0ahUKEwi3rPiWp737AhVZjLAFHSQJDAwQ4dUDCBA&uact=5&oq=einforma+B70341466&gs_lcp=Cgxnd3Mtd2l6LXNlcnAQAzIUCAAQ6gIQtAIQigMQtwMQ1AMQ5QIyFAgAEOoCELQCEIoDELcDENQDEOUCMhQIABDqAhC0AhCKAxC3AxDUAxDlAjIUCAAQ6gIQtAIQigMQtwMQ1AMQ5QIyFAgAEOoCELQCEIoDELcDENQDEOUCMhQIABDqAhC0AhCKAxC3AxDUAxDlAjIUCAAQ6gIQtAIQigMQtwMQ1AMQ5QIyFAgAEOoCELQCEIoDELcDENQDEOUCMhQIABDqAhC0AhCKAxC3AxDUAxDlAjIUCAAQ6gIQtAIQigMQtwMQ1AMQ5QJKBAhBGABKBAhGGABQoQhY7glgrApoAXAAeACAAQCIAQCSAQCYAQCgAQGwAQrAAQE&sclient=gws-wiz-serp')

In [ ]:
# box_e = []
# bot_link = []

In [11]:

def go_to_internet(cif, len_link, n_number):
    time.sleep(2)
    try:
        driver, wait = create_driver()
        driver.get('https://www.google.com/')
        try:
            wait.until(EC.presence_of_element_located((By.XPATH, f'//span[@class="ExCKkf z1asCe rzyADb"]')))
        except:
            pass
        
        c = search_google(driver, cif)
        if c:
            continue_with = click_page(driver, wait)
            print(f'Scraping {n_number}/{len_link}')

            if continue_with:
                bot_link.append((cif,continue_with))
                return (cif,continue_with)
            else:
                print('Failed')
            
    except Exception as ex:
        print(ex)
        traceback.print_exc(ex)
        return 
    
    finally:
        driver.close()


In [12]:
len_link = [len(list_cif)] * len(list_cif)

n_number = list(range(0, len(list_cif)))

In [16]:
len(bot_link)

0

In [22]:
# back_box = []
with futures.ThreadPoolExecutor(max_workers=3) as executor: # default/optimized number of threads
    box = list(executor.map(go_to_internet, list_cif[len(bot_link):], len_link, n_number))

Scraping 0/2576
Scraping 2/2576
Scraping 1/2576
Scraping 3/2576
Scraping 4/2576
Scraping 5/2576
Scraping 6/2576
Scraping 7/2576
Scraping 10/2576
Scraping 9/2576
Scraping 8/2576
Scraping 11/2576
Scraping 13/2576
Scraping 14/2576
Scraping 15/2576
Scraping 16/2576
Scraping 12/2576
Failed
Scraping 19/2576
Scraping 17/2576
Scraping 18/2576
Scraping 21/2576
Scraping 22/2576
Scraping 20/2576
HTTPConnectionPool(host='localhost', port=37481): Max retries exceeded with url: /session/e4718b3dec06555db87d0d1b31c6fb2f/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f651245a880>: Failed to establish a new connection: [Errno 111] Connection refused'))
HTTPConnectionPool(host='localhost', port=54417): Max retries exceeded with url: /session/b23fecf86f8ab312e3373ba4fee5f6e4/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f65122504f0>: Failed to establish a new connection: [Errno 111] Connection refused'))


KeyboardInterrupt: 

Scraping 24/2576


In [50]:
len(bot_link)

1276

In [15]:
bot_link = []

In [33]:
driver, wait = create_driver()
driver.get('https://www.google.com/')
for idx, cif in enumerate(list_cif):
    time.sleep(3)
    
    try:
        wait.until(EC.presence_of_element_located((By.XPATH, f'//span[@class="ExCKkf z1asCe rzyADb"]')))
    except:
        pass
        
    c = search_google(driver, cif)
    if c:
        continue_with = click_page(driver, wait)
        print(f'Scraping {idx}/{len(list_cif)}')

        if continue_with:
            bot_link.append((cif,continue_with))
        else:
            print('Failed')
    else:
        print('CAPTCHA')
        

Scraping 0/4272
Scraping 1/4272
Scraping 2/4272
Scraping 3/4272
Scraping 4/4272
Scraping 5/4272
Scraping 6/4272
Scraping 7/4272
Scraping 8/4272
Failed
Scraping 9/4272
Failed
Scraping 10/4272
Scraping 11/4272
Scraping 12/4272
Failed
Scraping 13/4272
Failed
Scraping 14/4272
Failed
Scraping 15/4272
Failed
Scraping 16/4272
Scraping 17/4272
Failed
Scraping 18/4272
Failed
Scraping 19/4272
Failed
Scraping 20/4272
Scraping 21/4272
Failed
Scraping 22/4272
Failed
Scraping 23/4272
Failed
Scraping 24/4272
Scraping 25/4272
Scraping 26/4272
Scraping 27/4272
Scraping 28/4272
Scraping 29/4272
Scraping 30/4272
Scraping 31/4272
Failed
Scraping 32/4272
Scraping 33/4272
Failed
Scraping 34/4272
Scraping 35/4272
Scraping 36/4272
Failed
Scraping 37/4272
Scraping 38/4272
Scraping 39/4272
Failed
Scraping 40/4272
Scraping 41/4272
Scraping 42/4272
Scraping 43/4272
Scraping 44/4272
Failed
Scraping 45/4272
Scraping 46/4272
Scraping 47/4272
Scraping 48/4272
Scraping 49/4272
Scraping 50/4272
Scraping 51/4272
Scrapin

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=105.0.5195.102)
Stacktrace:
#0 0x5557c140b693 <unknown>
#1 0x5557c1204b0a <unknown>
#2 0x5557c1207b57 <unknown>
#3 0x5557c12079ff <unknown>
#4 0x5557c120860a <unknown>
#5 0x5557c126f14a <unknown>
#6 0x5557c125a922 <unknown>
#7 0x5557c126e4b0 <unknown>
#8 0x5557c125a743 <unknown>
#9 0x5557c1230533 <unknown>
#10 0x5557c1231715 <unknown>
#11 0x5557c145b7bd <unknown>
#12 0x5557c145ebf9 <unknown>
#13 0x5557c1440f2e <unknown>
#14 0x5557c145f9b3 <unknown>
#15 0x5557c1434e4f <unknown>
#16 0x5557c147eea8 <unknown>
#17 0x5557c147f052 <unknown>
#18 0x5557c149971f <unknown>
#19 0x7f9a02afd609 <unknown>


In [34]:
with open('bot_link_g', 'wb') as file:
    pickle.dump(bot_link, file)

In [4]:
with open('bot_link_g', 'rb') as file:
    bot_link = pickle.load(file)

# PROCESS

## Leer

In [98]:
# with open('bot_link_7_12','wb') as file:
#     pickle.dump(bot_link, file)

Scraping 352/8245


In [5]:
with open('bot_link_7_12','rb') as file:
    bot_link = pickle.load(file)

## Execute

In [5]:
from concurrent import futures
from fake_useragent import UserAgent

In [6]:
def create_driver():
    ua = UserAgent()
    userAgent = ua.random
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
#     options.add_argument('--headless')
    options.add_argument("--incognito")
    options.add_argument(userAgent)
    prefs = {"profile.managed_default_content_settings.images": 2}
    options.add_experimental_option("prefs", prefs)
    capa = DesiredCapabilities.CHROME
    capa["pageLoadStrategy"] = "none"

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options, desired_capabilities=capa)
    wait = WebDriverWait(driver, 10)
    return driver, wait

In [7]:
def extract_info(driver, wait, link):
#     time.sleep(2.5)
    try:
        wait.until(EC.text_to_be_present_in_element((By.XPATH, '//tr[contains(.,"Domicilio social actual")]/td'),"Resultado de explotación"))
        wait.until(EC.presence_of_element_located((By.XPATH, f'//tr[contains(.,"Domicilio social actual")]/td')))
    except:
        pass
    try:

        try:
            driver.find_element('xpath', f'//*[contains(.,"Atención. Demasiadas peticiones detectadas")]')
        except NoSuchElementException:
               print('Problema con capcha')
        domicilio_social = driver.find_element('xpath', '//tr[contains(.,"Domicilio social actual")]/td').text.replace('Ver Mapa', '')
        
    except NoSuchElementException:
        domicilio_social = None
    if domicilio_social:
        print('OK')
    else:
        print('NO DATA')
#     try:
#         cif = driver.find_element('xpath', '//tr[contains(.,"CIF/NIF")]/td').text
#         print(cif)
#     except NoSuchElementException:
#         cif = None
    cif = link[0]
    try:
        postal_code = driver.find_element('xpath', '//tr[contains(.,"Código Postal")]/td').text
    except NoSuchElementException:
        postal_code = None
    try:
        municipio = driver.find_element('xpath', '//tr[contains(.,"Municipio")]/td').text
    except NoSuchElementException:
        municipio = None
    try:
        provincia = driver.find_element('xpath', '//tr[contains(.,"Provincia")]/td').text
    except NoSuchElementException:
        provincia = None
    try:
        donominacion_antigua = driver.find_element('xpath', '//tr[contains(.,"Denominación  Antigua")]/td').text
    except NoSuchElementException:
        donominacion_antigua = None
    try:
        web = driver.find_element('xpath', '//tr[contains(.,"Web")]/td').text
    except:
        web = None
    try:
        canae = driver.find_element('xpath', '//tr[contains(.,"CNAE ")]/td').text
    except NoSuchElementException:
        canae = None
    try:
        fecha_actualizacion_ficha = driver.find_element('xpath', '//tr[contains(.,"Actualización Ficha Empresa")]/td').text
    except NoSuchElementException:
        fecha_actualizacion_ficha = None
    try:
        capital_social = driver.find_element('xpath', '//tr[contains(.,"Capital Social")]/td').text
    except NoSuchElementException:
        capital_social = None
    try:
        cotiza_en_bolza = driver.find_element('xpath', '//tr[contains(.,"Cotiza en Bolsa")]/td').text
    except NoSuchElementException:
        cotiza_en_bolza = None
    try:
        resultado_ultimo_year = driver.find_element('xpath', '//tr[contains(.,"Resultado Último Año")]/td').text
    except NoSuchElementException:
        resultado_ultimo_year = None
    try:
        posicion_en_ranking = driver.find_element('xpath', '//p[@class="textcenter"]/span[@class="big"]').text
    except:
        posicion_en_ranking = None
    try:
        tendencia_ranking = driver.find_element('xpath', '//div[@class="mod-border_rankings color01_sellotop"]//p[@class="mt5 textcenter"]/span/span').get_attribute('class')
    except:
        tendencia_ranking = '-'
    if tendencia_ranking == 'flecha-rating mayor':
        tendencia_ranking = 'Alta'
    else:
        tendencia_ranking = 'Baja'
    try:
        numero_empleados = driver.find_element('xpath', '//tr[contains(.,"Número de Empleados")]/td').text
    except NoSuchElementException:
        numero_empleados = None
    try:
        actividades_internacionales = driver.find_element('xpath', '//tr[contains(.,"Actividades Internacionales")]/td').text
    except:
        actividades_internacionales = None
    
    try:
        depositio_mercantil = driver.find_element('xpath', '//tr[contains(.,"Último Depósito disponible en el Registro Mercantil")]/td').text
    except NoSuchElementException:
        depositio_mercantil = None
        
    try:
        cargos_directivos = [i.text.replace('\nConsultar', '') for i in driver.find_elements('xpath', '//div[@id="ancla_directivos_funcionales"]//tbody/tr')]
    except:
        cargos_directivos = None
    driver.execute_script("window.stop();")
    
    return {
        'Cif':cif,
        'Domicilio Social': domicilio_social,
        'Codigo Postal': postal_code,
        'Municipio':municipio,
        'Provincia':provincia,
        'Denominacion_antigua':donominacion_antigua,
        'Web':web,
        'CANAE 2009': canae,
        'Fecha Actualizacion Ficha Empresa':fecha_actualizacion_ficha,
        'Capital Social': capital_social,
        'Cotiza en Bolsa': cotiza_en_bolza,
        'Resultado Ultimo Año': resultado_ultimo_year,
        'Posicion Ranking': posicion_en_ranking,
        'Tendencia Ranking': tendencia_ranking,
        'Numero empleados': numero_empleados,
        'Actividad Internacional': actividades_internacionales,
        'Último	deposito registro mercantil': depositio_mercantil,
        'Cargos Directivos': cargos_directivos
    }
    
    

In [8]:
def take_screenshot(link, len_link, n_number):
    url = link[-1]
    try:
        print(f'Scraping {n_number}/{len_link}')
        
        driver, wait = create_driver()
        driver.get(url)
        try:
            wait.until(EC.text_to_be_present_in_element((By.XPATH, '//tr[contains(.,"Domicilio social actual")]/td'),"Resultado de explotación"))
            
        except:
            pass
        
        data = extract_info(driver, wait, link)
        clear_output(wait=True)
        back_box.append(data)
        return data
        
    
    except Exception as ex:
        print(ex)
        traceback.print_exc(ex)
        return 
    
    finally:
        driver.close()

In [9]:
len_link = [len(bot_link)] * len(bot_link)

n_number = list(range(0, len(bot_link)))

In [10]:
back_box = []
with futures.ThreadPoolExecutor(max_workers=5) as executor: # default/optimized number of threads
    box = list(executor.map(take_screenshot, bot_link, len_link, n_number))

Problema con capcha
OK


In [12]:
len(back_box)

6067

In [14]:
with open('back_box_f', 'wb') as file:
    pickle.dump(back_box, file)

In [149]:
back_box[::-1]

[{'Cif': None,
  'Domicilio Social': None,
  'Codigo Postal': None,
  'Municipio': None,
  'Provincia': None,
  'Denominacion_antigua': None,
  'Web': None,
  'CANAE 2009': None,
  'Fecha Actualizacion Ficha Empresa': None,
  'Capital Social': None,
  'Cotiza en Bolsa': None,
  'Resultado Ultimo Año': None,
  'Posicion Ranking': None,
  'Tendencia Ranking': 'Baja',
  'Numero empleados': None,
  'Actividad Internacional': None,
  'Último\tdeposito registro mercantil': None,
  'Cargos Directivos': []},
 {'Cif': None,
  'Domicilio Social': None,
  'Codigo Postal': None,
  'Municipio': None,
  'Provincia': None,
  'Denominacion_antigua': None,
  'Web': None,
  'CANAE 2009': None,
  'Fecha Actualizacion Ficha Empresa': None,
  'Capital Social': None,
  'Cotiza en Bolsa': None,
  'Resultado Ultimo Año': None,
  'Posicion Ranking': None,
  'Tendencia Ranking': 'Baja',
  'Numero empleados': None,
  'Actividad Internacional': None,
  'Último\tdeposito registro mercantil': None,
  'Cargos Direct

In [15]:
df = pd.DataFrame(back_box).dropna(subset=['Cif'])

In [18]:
df

,Cif,Domicilio Social,Codigo Postal,Municipio,Provincia,Denominacion_antigua,Web,CANAE 2009,Fecha Actualizacion Ficha Empresa,Capital Social,Cotiza en Bolsa,Resultado Ultimo Año,Posicion Ranking,Tendencia Ranking,Numero empleados,Actividad Internacional,Último\tdeposito registro mercantil,Cargos Directivos
0,A15234065,AVENIDA DEPUTACION (POL. IND. SABON EDF INDITE...,15142,ARTEIXO,La Coruña,None,www.tempe.es,4642 - Comercio al por mayor de prendas de ves...,07/12/2022,"60.101,21 €",NO,Positivo Consultar,138,Baja,1918 (año 2021),Importa y Exporta,2021,"[Director General ANTONIO G..., Director Gener..."
1,A81473852,"CALLE LA CONSTITUCIO, 1 - 1º PLANTA",8960,SANT JUST DESVERN,Barcelona,MOYRESA MOLTURACION Y REFINO S.A.,www.bungeiberica.com\nwww.bunge.com,"4621 - Comercio al por mayor de cereales, taba...",10/12/2022,66.171.000 €,NO,Positivo Consultar,44,Alta,326 (año 2022),Importa y Exporta,2021,"[Director/Responsable RRHH LAURA G..., Directo..."
2,A78276854,"AVENIDA DEPUTACION (PG IND EL SABON), S/N - ED...",15142,ARTEIXO,La Coruña,IVAN SA,www.bershka.com,4642 - Comercio al por mayor de prendas de ves...,07/12/2022,6.010.121 €,NO,Positivo Consultar,127,Baja,3118 (año 2021),Importa y Exporta,2021,"[Director General ANTONIO F..., Director Gener..."
3,A15075062,AVENIDA DEPUTACION (PG IND DE SABON) - EDIF. I...,15142,ARTEIXO,La Coruña,None,www.inditex.com,4642 - Comercio al por mayor de prendas de ves...,09/12/2022,93.499.560 €,SI,Positivo Consultar,7,Baja,2536 (año 2022),Importa y Exporta,2021,"[Director/Responsable Financiero IGNACIO F...,..."
4,A15022510,"AVENIDA DEPUTACION (EDIF. INDITEX), S/N",15142,ARTEIXO,La Coruña,GOASAM SL\nGOASAM SA,www.inditex.com,4771 - Comercio al por menor de prendas de ves...,09/12/2022,1.534.174 €,NO,Negativo Consultar,94,Baja,10730 (año 2021),No constan,2021,"[Director General OSCAR P..., Director General..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6062,B70520622,AVENIDA PRIMERA (POLIGONO POCOMACO) (POL. DE P...,15190,A CORUÑA,La Coruña,None,None,4652 - Comercio al por mayor de equipos electr...,25/11/2022,3.200 €,NO,Negativo Consultar,None,Baja,4 (año 2019),No constan,2021,[]
6063,B15224603,"CALLE SAN ROSENDO, 45 - B",15007,A CORUÑA,La Coruña,None,None,3319 - Reparación de otros equipos,20/10/2022,None,NO,None,None,Baja,2 (año 2008),No constan,2004,"[Gerente ARTURO M..., Director/Responsable Fin..."
6064,B70074372,"CALLE EUROPA, 20 - BJ",15007,A CORUÑA,La Coruña,None,None,3312 - Reparación de maquinaria,23/11/2022,3.100 €,NO,Positivo Consultar,370.221,Alta,2 (año 2021),No constan,2021,[]
6065,A15062953,"AVENIDA EJERCITO, 42 - BJ",15006,A CORUÑA,La Coruña,None,None,9200 - Actividades de juegos de azar y apuestas,02/12/2022,113.643 €,NO,Negativo Consultar,311.911,Baja,10 (año 2021),No constan,2021,"[Gerente MANUEL F..., Director/Responsable Fin..."


In [48]:
box_df = []
for idx, link in enumerate(bot_link[2000:]):
#     time.sleep(3)
    driver.get(link)
    print(f'Scraping {idx+4020}/{len(bot_link)-1}')
    
    #CAMPOS
    
    
    
    try:
        try:
            wait.until(EC.presence_of_element_located((By.XPATH, f'//tr[contains(.,"Domicilio social actual")]/td')))
        except TimeoutException:
            try:
                driver.find_element('xpath', f'//*[contains(.,"Atención. Demasiadas peticiones detectadas")]')
                input('Dame ')
                break
            except NoSuchElementException:
                pass
            print(f'error en la ur: {cif}')
        domicilio_social = driver.find_element('xpath', '//tr[contains(.,"Domicilio social actual")]/td').text.replace('Ver Mapa', '')
        
    except NoSuchElementException:
        domicilio_social = None
    try:
        cif = driver.find_element('xpath', '//tr[contains(.,"CIF/NIF")]/td').text
    except NoSuchElementException:
        cif = None
    try:
        postal_code = driver.find_element('xpath', '//tr[contains(.,"Código Postal")]/td').text
    except NoSuchElementException:
        postal_code = None
    try:
        municipio = driver.find_element('xpath', '//tr[contains(.,"Municipio")]/td').text
    except NoSuchElementException:
        municipio = None
    try:
        provincia = driver.find_element('xpath', '//tr[contains(.,"Provincia")]/td').text
    except NoSuchElementException:
        provincia = None
    try:
        donominacion_antigua = driver.find_element('xpath', '//tr[contains(.,"Denominación  Antigua")]/td').text
    except NoSuchElementException:
        donominacion_antigua = None
    try:
        web = driver.find_element('xpath', '//tr[contains(.,"Web")]/td').text
    except:
        web = None
    try:
        canae = driver.find_element('xpath', '//tr[contains(.,"CNAE ")]/td').text
    except NoSuchElementException:
        canae = None
    try:
        fecha_actualizacion_ficha = driver.find_element('xpath', '//tr[contains(.,"Actualización Ficha Empresa")]/td').text
    except NoSuchElementException:
        fecha_actualizacion_ficha = None
    try:
        capital_social = driver.find_element('xpath', '//tr[contains(.,"Capital Social")]/td').text
    except NoSuchElementException:
        capital_social = None
    try:
        cotiza_en_bolza = driver.find_element('xpath', '//tr[contains(.,"Cotiza en Bolsa")]/td').text
    except NoSuchElementException:
        cotiza_en_bolza = None
    try:
        resultado_ultimo_year = driver.find_element('xpath', '//tr[contains(.,"Resultado Último Año")]/td').text
    except NoSuchElementException:
        resultado_ultimo_year = None
    try:
        posicion_en_ranking = driver.find_element('xpath', '//p[@class="textcenter"]/span[@class="big"]').text
    except:
        posicion_en_ranking = None
    try:
        tendencia_ranking = driver.find_element('xpath', '//div[@class="mod-border_rankings color01_sellotop"]//p[@class="mt5 textcenter"]/span/span').get_attribute('class')
    except:
        tendencia_ranking = '-'
    if tendencia_ranking == 'flecha-rating mayor':
        tendencia_ranking = 'Alta'
    else:
        tendencia_ranking = 'Baja'
    try:
        numero_empleados = driver.find_element('xpath', '//tr[contains(.,"Número de Empleados")]/td').text
    except NoSuchElementException:
        numero_empleados = None
    try:
        actividades_internacionales = driver.find_element('xpath', '//tr[contains(.,"Actividades Internacionales")]/td').text
    except:
        actividades_internacionales = None
    
    try:
        depositio_mercantil = driver.find_element('xpath', '//tr[contains(.,"Último Depósito disponible en el Registro Mercantil")]/td').text
    except NoSuchElementException:
        depositio_mercantil = None
        
    try:
        cargos_directivos = [i.text.replace('\nConsultar', '') for i in driver.find_elements('xpath', '//div[@id="ancla_directivos_funcionales"]//tbody/tr')]
    except:
        cargos_directivos = None
    
    
    box_df.append({
        'Cif':cif,
        'Domicilio Social': domicilio_social,
        'Codigo Postal': postal_code,
        'Municipio':municipio,
        'Provincia':provincia,
        'Denominacion_antigua':donominacion_antigua,
        'Web':web,
        'CANAE 2009': canae,
        'Fecha Actualizacion Ficha Empresa':fecha_actualizacion_ficha,
        'Capital Social': capital_social,
        'Cotiza en Bolsa': cotiza_en_bolza,
        'Resultado Ultimo Año': resultado_ultimo_year,
        'Posicion Ranking': posicion_en_ranking,
        'Tendencia Ranking': tendencia_ranking,
        'Numero empleados': numero_empleados,
        'Actividad Internacional': actividades_internacionales,
        'Último	deposito registro mercantil': depositio_mercantil,
        'Cargos Directivos': cargos_directivos
    })
    
    

Scraping 4020/8244
Scraping 4021/8244
Scraping 4022/8244
Scraping 4023/8244
Scraping 4024/8244
Scraping 4025/8244
error en la ur: B157...
Scraping 4026/8244
Scraping 4027/8244
Scraping 4028/8244
Scraping 4029/8244
Scraping 4030/8244
Scraping 4031/8244
error en la ur: None
Scraping 4032/8244
Scraping 4033/8244
Scraping 4034/8244
Scraping 4035/8244
Scraping 4036/8244
Scraping 4037/8244
Scraping 4038/8244
Scraping 4039/8244
Scraping 4040/8244
Scraping 4041/8244
Scraping 4042/8244
Scraping 4043/8244
Scraping 4044/8244
Scraping 4045/8244
Scraping 4046/8244
Scraping 4047/8244
Scraping 4048/8244
Scraping 4049/8244
Scraping 4050/8244
error en la ur: B158...
Scraping 4051/8244
Scraping 4052/8244
Scraping 4053/8244
Scraping 4054/8244
Scraping 4055/8244
Scraping 4056/8244
Scraping 4057/8244
Scraping 4058/8244
Scraping 4059/8244
Scraping 4060/8244
Scraping 4061/8244
Scraping 4062/8244
Scraping 4063/8244
Scraping 4064/8244
Scraping 4065/8244
Scraping 4066/8244
Scraping 4067/8244
Scraping 4068/8244


Dame  j


In [19]:
df.to_csv('Data_einforma_last', index=False)